# Common mistakes

## 1. Creating cells without `cell` decorator

The cell decorator names cells deterministically and uniquely based on the name of the functions and its parameters.

It also uses a caching mechanisms that improves performance and guards against duplicated names.

### 1.a naming cells manually

Naming cells manually is susceptible to name colisions

in GDS you can't have two cells with the same name.

For example: this code will fail

```python

import gdsfactory as gf

c1 = gf.Component('wg')
c1 << gf.components.straight(length = 5)


c2 = gf.Component('wg')
c2 << gf.components.straight(length = 50)


c3 = gf.Component('waveguides')
wg1 = c3 << c1
wg2 = c3 << c2
wg2.movey(10)
c3

```

**Solution**: Use the `gf.cell` decorator

In [ ]:
import gdsfactory as gf


@gf.cell
def wg(length: float = 3):
    return gf.components.straight(length=length)


print(wg(length=5))
print(wg(length=50))

### 1.b Not naming components with a unique and deterministic name

In the case of not wrapping the function with `cell` you will get unique names thanks to the unique identifier `uuid`.

This name will be different and non-deterministic for different invocations of the script.

However it will be hard for you to know where that cell came from.

Notice how gdsfactory raises a Warning


In [ ]:
c1 = gf.Component()
c2 = gf.Component()

print(c1.name)
print(c2.name)

### 1.c Intermediate Unnamed cells

While creating a cell, you should not create intermediate cells, because they won't get a name.

In [ ]:
@gf.cell
def die_bad():
    """c will be an intermediate unnamed cell"""
    c = gf.Component()
    c << gf.components.straight(length=10)
    c2 = gf.components.die_bbox(c, street_width=10)
    return c2


c = die_bad(cache=False)
print(c.references)
c.show()
c.plot()

**Solution1** Don't use intermediate cells

In [ ]:
@gf.cell
def die_good():
    c = gf.Component()
    c << gf.components.straight(length=10)
    c << gf.components.die_bbox_frame(c.bbox, street_width=10)
    return c


c = die_good(cache=False)
print(c.references)
c.show()
c.plot()

**Solution2** You can flatten the cell, but you will lose the memory savings from cell references. Solution1 is more elegant.

In [ ]:
@gf.cell
def die_flat():
    """c will be an intermediate unnamed cell"""
    c = gf.Component()
    c << gf.components.straight(length=10)
    c2 = gf.components.die_bbox(c, street_width=10)
    c2 = c2.flatten()
    return c2


c = die_flat(cache=False)
print(c.references)
c.show()
c.plot()